# ***TASK 2***

Description:

"The file “Parts.csv” contains descriptions of some fictitious parts. Your goal is to find 5 alternative parts to each provided fictitious part in the dataset based on their similarity. First provide descriptive analysis of the data and highlight 2-3 findings and difficulties of the data that we provided and describe how you would handle this. Continue to implement a model that is finding the similar fictitious parts based on the column “DESCRIPTION”. Please give details of your model and why you choose this. Once you finished your implementation of your chosen model, please think about how you would change or addition your code, when you have more entries and would consider the additional attributes."

In [10]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import OneHotEncoder

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', None)

## Read data

In [2]:
df = pd.read_csv('./data/parts.csv', sep=';')

In [3]:
df.shape

(998, 32)

In [4]:
df.head()

,ID,DESCRIPTION,Attribut1,Additional Feature,Application,Characteristic,Temp,Height,Length in mm,Rating,Material,Size,Code,Joule-integral-Nom (J),LC Risk,Maximum AC Voltage Rating,Maximum DC Voltage Rating,Maximum Power Dissipation,Mounting,Mounting Feature,Number of Terminals,Operating Temperature-Max (Cel),Operating Temperature-Min (Cel),Physical Dimension,Pre-arcing time-Min (ms),Product Diameter,Product Length,Rated Breaking Capacity (A),Rated Current (A),Rated Voltage (V),Rated Voltage(AC) (V),Rated Voltage(DC) (V)
0,A1,"Indicator Red Fast Movement 1.6A 250V Holder Plastic 5 X 20mm Ceramic Box CCC/PSE/VDE/cULus Electric Indicator, Very Fast Blow, 1.6A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,1.6A,Ceramic,5 X 20mm,NaN,0.755J,Low,250V,NaN,4W,Holder,INLINE/HOLDER,2.0,125Cel,-55Cel,5.2mm x 20mm,3ms,5.2mm,20mm,1500A,1.6A,250V,250V,NaN
1,A2,"Non Resettable Indicators Electric Indicator, Very Fast Blow, 6.3A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",NaN,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,NaN,Ceramic,5 X 20mm,e2,36.7J,Low,NaN,NaN,NaN,Holder,INLINE/HOLDER,2.0,125Cel,-55Cel,5.2mm x 20mm,3ms,5.2mm,20mm,1500A,6.3A,250V,250V,NaN
2,A3,"Indicator Red Fast Movement 8A 250V Holder Plastic 5 X 20mm Ceramic Box KC/PSE/VDE/cULus Electric Indicator, Very Fast Blow, 8A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,8A,Ceramic,5 X 20mm,e2,81.9J,Low,250V,NaN,4W,Holder,INLINE/HOLDER,2.0,125Cel,-55Cel,5.2mm x 20mm,10ms,5.2mm,20mm,1500A,8A,250V,250V,NaN
3,A4,"Non Resettable Indicators Electric Indicator, Very Fast Blow, 10A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",NaN,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,NaN,Ceramic,5 X 20mm,e2,141J,Low,NaN,NaN,NaN,Holder,INLINE/HOLDER,2.0,125Cel,-55Cel,5.2mm x 20mm,10ms,5.2mm,20mm,1500A,10A,250V,250V,NaN
4,A5,"Indicator Red Fast Movement 12.5A 250V Holder Plastic 5 X 20mm Ceramic Box PSE/cULus Electric Indicator, Very Fast Blow, 12.5A, 250VAC, 500A (IR), Inline/holder, 5x20mm",Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,12.5A,Ceramic,5 X 20mm,NaN,203J,Low,250V,NaN,6.9(Typ)W,Holder,INLINE/HOLDER,2.0,125Cel,-55Cel,5.2mm x 20mm,10ms,5.2mm,20mm,500A,12.5A,250V,250V,NaN


## Preprocess & EDA

In the following we examine some difficulties with using the raw data for modeling and apply some solutions.

### Difficulty 1: Columns with mixed data types

Immediately, one notices that the **majority of columns in `parts.csv` are object type**, which, in Pandas speak, means they contain mixed data types (e.g. numeric, string, and/or possibly missing). **ML models cannot directly work with non-numeric values; thus, these values must be first converted to a representation before modeling can proceed**. Examining the head of the file, we see that **many (but not all) of these columns contain a numeric value and a string, most commonly the measurement type (e.g. mm, W, V, A)**. This string measurement information is frequently already stored in the feature name, so we can discard this information, converting it to a numeric feature.

In [5]:
df.dtypes

ID                                  object
DESCRIPTION                         object
Attribut1                           object
Additional Feature                  object
Application                         object
Characteristic                      object
Temp                                object
Height                              object
Length in mm                        object
Rating                              object
Material                            object
Size                                object
Code                                object
Joule-integral-Nom (J)              object
LC Risk                             object
Maximum AC Voltage Rating           object
Maximum DC Voltage Rating           object
Maximum Power Dissipation           object
Mounting                            object
Mounting Feature                    object
Number of Terminals                float64
Operating Temperature-Max (Cel)     object
Operating Temperature-Min (Cel)     object
Physical Di

In [6]:
df.head()

,ID,DESCRIPTION,Attribut1,Additional Feature,Application,Characteristic,Temp,Height,Length in mm,Rating,Material,Size,Code,Joule-integral-Nom (J),LC Risk,Maximum AC Voltage Rating,Maximum DC Voltage Rating,Maximum Power Dissipation,Mounting,Mounting Feature,Number of Terminals,Operating Temperature-Max (Cel),Operating Temperature-Min (Cel),Physical Dimension,Pre-arcing time-Min (ms),Product Diameter,Product Length,Rated Breaking Capacity (A),Rated Current (A),Rated Voltage (V),Rated Voltage(AC) (V),Rated Voltage(DC) (V)
0,A1,"Indicator Red Fast Movement 1.6A 250V Holder Plastic 5 X 20mm Ceramic Box CCC/PSE/VDE/cULus Electric Indicator, Very Fast Blow, 1.6A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,1.6A,Ceramic,5 X 20mm,NaN,0.755J,Low,250V,NaN,4W,Holder,INLINE/HOLDER,2.0,125Cel,-55Cel,5.2mm x 20mm,3ms,5.2mm,20mm,1500A,1.6A,250V,250V,NaN
1,A2,"Non Resettable Indicators Electric Indicator, Very Fast Blow, 6.3A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",NaN,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,NaN,Ceramic,5 X 20mm,e2,36.7J,Low,NaN,NaN,NaN,Holder,INLINE/HOLDER,2.0,125Cel,-55Cel,5.2mm x 20mm,3ms,5.2mm,20mm,1500A,6.3A,250V,250V,NaN
2,A3,"Indicator Red Fast Movement 8A 250V Holder Plastic 5 X 20mm Ceramic Box KC/PSE/VDE/cULus Electric Indicator, Very Fast Blow, 8A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,8A,Ceramic,5 X 20mm,e2,81.9J,Low,250V,NaN,4W,Holder,INLINE/HOLDER,2.0,125Cel,-55Cel,5.2mm x 20mm,10ms,5.2mm,20mm,1500A,8A,250V,250V,NaN
3,A4,"Non Resettable Indicators Electric Indicator, Very Fast Blow, 10A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",NaN,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,NaN,Ceramic,5 X 20mm,e2,141J,Low,NaN,NaN,NaN,Holder,INLINE/HOLDER,2.0,125Cel,-55Cel,5.2mm x 20mm,10ms,5.2mm,20mm,1500A,10A,250V,250V,NaN
4,A5,"Indicator Red Fast Movement 12.5A 250V Holder Plastic 5 X 20mm Ceramic Box PSE/cULus Electric Indicator, Very Fast Blow, 12.5A, 250VAC, 500A (IR), Inline/holder, 5x20mm",Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,12.5A,Ceramic,5 X 20mm,NaN,203J,Low,250V,NaN,6.9(Typ)W,Holder,INLINE/HOLDER,2.0,125Cel,-55Cel,5.2mm x 20mm,10ms,5.2mm,20mm,500A,12.5A,250V,250V,NaN


#### Solution: Strip strings from select mixed-type columns

*Note* - I'm unsure how to handle strange values such as '16@(CSA/UL)|10@VDEA' in `Rating`, so I treat them the same as other values. In the aforementioned case, this would reduce to 16. Obviously in real world applications you'd want to understand the data better.

Further, whereas it would also be possible to convert columns such as `Size` and `Physical Dimension` to numeric representations (e.g. area, volume), I treat these as categorical (see below). This allows for the most flexibility when modeling using these columns.

In [7]:
cols = ['Temp', 'Height', 'Length in mm', 'Rating', 'Joule-integral-Nom (J)', 'Maximum AC Voltage Rating',
        'Maximum DC Voltage Rating', 'Maximum Power Dissipation', 'Operating Temperature-Max (Cel)',
        'Operating Temperature-Min (Cel)', 'Pre-arcing time-Min (ms)', 'Product Diameter',
        'Product Length', 'Rated Breaking Capacity (A)', 'Rated Current (A)', 'Rated Voltage (V)',
        'Rated Voltage(AC) (V)', 'Rated Voltage(DC) (V)']

In [8]:
for col in cols:
    df[col] = df[col].apply(lambda x: float(re.findall(r"[-+]?(?:\d*\.*\d+)", x)[0]) if not pd.isnull(x) else x)
assert [np.issubdtype(df[i].dtype, np.number) for i in cols]  # verify all numeric

In [21]:
df.head(3)

,ID,DESCRIPTION,Attribut1,Additional Feature,Application,Characteristic,Temp,Height,Length in mm,Rating,Material,Size,Code,Joule-integral-Nom (J),LC Risk,Maximum AC Voltage Rating,Maximum DC Voltage Rating,Maximum Power Dissipation,Mounting,Mounting Feature,Number of Terminals,Operating Temperature-Max (Cel),Operating Temperature-Min (Cel),Physical Dimension,Pre-arcing time-Min (ms),Product Diameter,Product Length,Rated Breaking Capacity (A),Rated Current (A),Rated Voltage (V),Rated Voltage(AC) (V),Rated Voltage(DC) (V)
0,A1,"Indicator Red Fast Movement 1.6A 250V Holder Plastic 5 X 20mm Ceramic Box CCC/PSE/VDE/cULus Electric Indicator, Very Fast Blow, 1.6A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20.0,5.2,1.6,Ceramic,5 X 20mm,NaN,0.755,Low,250.0,NaN,4.0,Holder,INLINE/HOLDER,2.0,125.0,-55.0,5.2mm x 20mm,3.0,5.2,20.0,1500.0,1.6,250.0,250.0,NaN
1,A2,"Non Resettable Indicators Electric Indicator, Very Fast Blow, 6.3A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",NaN,NaN,Primary Protection In Equipment,VERY FAST,NaN,20.0,5.2,NaN,Ceramic,5 X 20mm,e2,36.700,Low,NaN,NaN,NaN,Holder,INLINE/HOLDER,2.0,125.0,-55.0,5.2mm x 20mm,3.0,5.2,20.0,1500.0,6.3,250.0,250.0,NaN
2,A3,"Indicator Red Fast Movement 8A 250V Holder Plastic 5 X 20mm Ceramic Box KC/PSE/VDE/cULus Electric Indicator, Very Fast Blow, 8A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20.0,5.2,8.0,Ceramic,5 X 20mm,e2,81.900,Low,250.0,NaN,4.0,Holder,INLINE/HOLDER,2.0,125.0,-55.0,5.2mm x 20mm,10.0,5.2,20.0,1500.0,8.0,250.0,250.0,NaN


### Difficulty 2: Catgorical features

Several of the features in `parts.csv` are purely categorical (e.g. `Attribut1`, `Characteristic`). These cannot be directly incorporated in a ML model either.

#### Solution: As in Task 1, I convert categorical features to one-hot representations

*Note* - We exclude the feature, `ID`, since this is unique for each row. Similarity, we also exclude `DESCRIPTION`, since this is the feature we intend to identify outliers in.

In [17]:
categorical_cols = [i for i in df.columns if i not in cols and i not in ['ID', 'DESCRIPTION']]
df[categorical_cols].nunique(dropna=False)

Attribut1               6
Additional Feature     40
Application            29
Characteristic          8
Material                9
Size                   40
Code                    5
LC Risk                 4
Mounting                6
Mounting Feature        4
Number of Terminals     2
Physical Dimension     39
dtype: int64

In [18]:
X = OneHotEncoder().fit_transform(df[categorical_cols]).toarray()
print(X.shape)

(998, 192)


In [19]:
categorical_cols

['Attribut1',
 'Additional Feature',
 'Application',
 'Characteristic',
 'Material',
 'Size',
 'Code',
 'LC Risk',
 'Mounting',
 'Mounting Feature',
 'Number of Terminals',
 'Physical Dimension']

In [20]:
df.head(3)

,ID,DESCRIPTION,Attribut1,Additional Feature,Application,Characteristic,Temp,Height,Length in mm,Rating,Material,Size,Code,Joule-integral-Nom (J),LC Risk,Maximum AC Voltage Rating,Maximum DC Voltage Rating,Maximum Power Dissipation,Mounting,Mounting Feature,Number of Terminals,Operating Temperature-Max (Cel),Operating Temperature-Min (Cel),Physical Dimension,Pre-arcing time-Min (ms),Product Diameter,Product Length,Rated Breaking Capacity (A),Rated Current (A),Rated Voltage (V),Rated Voltage(AC) (V),Rated Voltage(DC) (V)
0,A1,"Indicator Red Fast Movement 1.6A 250V Holder Plastic 5 X 20mm Ceramic Box CCC/PSE/VDE/cULus Electric Indicator, Very Fast Blow, 1.6A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20.0,5.2,1.6,Ceramic,5 X 20mm,NaN,0.755,Low,250.0,NaN,4.0,Holder,INLINE/HOLDER,2.0,125.0,-55.0,5.2mm x 20mm,3.0,5.2,20.0,1500.0,1.6,250.0,250.0,NaN
1,A2,"Non Resettable Indicators Electric Indicator, Very Fast Blow, 6.3A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",NaN,NaN,Primary Protection In Equipment,VERY FAST,NaN,20.0,5.2,NaN,Ceramic,5 X 20mm,e2,36.700,Low,NaN,NaN,NaN,Holder,INLINE/HOLDER,2.0,125.0,-55.0,5.2mm x 20mm,3.0,5.2,20.0,1500.0,6.3,250.0,250.0,NaN
2,A3,"Indicator Red Fast Movement 8A 250V Holder Plastic 5 X 20mm Ceramic Box KC/PSE/VDE/cULus Electric Indicator, Very Fast Blow, 8A, 250VAC, 1500A (IR), Inline/holder, 5x20mm",Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20.0,5.2,8.0,Ceramic,5 X 20mm,e2,81.900,Low,250.0,NaN,4.0,Holder,INLINE/HOLDER,2.0,125.0,-55.0,5.2mm x 20mm,10.0,5.2,20.0,1500.0,8.0,250.0,250.0,NaN


In [ ]:
df['DESCRIPTION'].head()

In [ ]:
df['DESCRIPTION'].duplicated().sum()

In [ ]:
dup_bool = df['DESCRIPTION'].duplicated()

In [ ]:
foo = df[dup_bool][['DESCRIPTION']]

In [ ]:
foo = foo[foo['DESCRIPTION'].notnull()]

In [ ]:
foo = foo.sort_values(by=['DESCRIPTION'])

In [ ]:
foo

In [ ]:
len(foo)